# basic gnn
---
[TOC]

In [ ]:
# Load the alexandria, model
import mylib.datasets.alexandria as alex
import mylib.models.basic_gnn as gnn

# Load the dataset
dataset = alex.Dataset(root="/home/gengyao/matbench", partial=(0.0,0.05), tensorize_work=True)
# Initialize the model
model = gnn.Model()
print(dataset, model)

## model training

In [ ]:
import torch as tc
import torch.nn as nn
from torch.optim import SGD

optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
loss_fn = nn.L1Loss()
batch_size = 4
batch_size_cnt = 0
loss = 0
# training loop
for epoch in range(100):
    for sample in dataset:
        y_bar = tc.tensor([[sample[0], sample[1]]]).to(model.device)
        y = model.forward(sample[3], sample[4], sample[5])
        loss += loss_fn(y, y_bar)
        batch_size_cnt += 1
        if batch_size_cnt == batch_size:
            # backpropagation
            print(f"epoch:{epoch}, loss:{loss}")
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss = 0
            batch_size_cnt = 0

## model inference

In [ ]:
# TODO: model inference
